Assignment: Notebook for Graded Assessment

In [8]:
!pip install ipython-sql prettytable

import prettytable

prettytable.DEFAULT = 'DEFAULT'

Store the datasets in database tables
To analyze the data using SQL, it first needs to be loaded into SQLite DB. We will create three tables in as under:

CENSUS_DATA
CHICAGO_PUBLIC_SCHOOLS
CHICAGO_CRIME_DATA
Load the pandas and sqlite3 libraries and establish a connection to FinalDB.db

In [2]:
import pandas as pd
import sqlite3

# Establish connection to the SQLite database
conn = sqlite3.connect("FinalDB.db")

# Create a cursor object (optional, if executing raw SQL queries)
cursor = conn.cursor()

print("Connection to FinalDB.db established successfully!")


Connection to FinalDB.db established successfully!


In [ ]:
Load the SQL magic module

In [9]:
%load_ext sql
%sql sqlite:///FinalDB.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Use Pandas to load the data available in the links above to dataframes. Use these dataframes to load data on to the database FinalDB.db as required tables.

In [10]:
import pandas as pd
import sqlite3

# Load datasets from CSV files into Pandas DataFrames
census_data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
chicago_schools = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
chicago_crime = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")

# Connect to FinalDB.db 
conn = sqlite3.connect("FinalDB.db")

# Save data into the SQLite database as tables
census_data.to_sql("CENSUS_DATA", conn, if_exists="replace", index=False)
chicago_schools.to_sql("CHICAGO_PUBLIC_SCHOOLS", conn, if_exists="replace", index=False)
chicago_crime.to_sql("CHICAGO_CRIME_DATA", conn, if_exists="replace", index=False)

# Create a cursor object
cursor = conn.cursor()

# Check the tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())  # Should list the table names

# Fetch first 5 rows from CHICAGO_CRIME_DATA
print(pd.read_sql("SELECT * FROM CHICAGO_CRIME_DATA LIMIT 5", conn))


[('CENSUS_DATA',), ('CHICAGO_PUBLIC_SCHOOLS',), ('CHICAGO_CRIME_DATA',)]
         ID CASE_NUMBER        DATE                     BLOCK IUCR  \
0   3512276    HK587712  2004-08-28        047XX S KEDZIE AVE  890   
1   3406613    HK456306  2004-06-26  009XX N CENTRAL PARK AVE  820   
2   8002131    HT233595  2011-04-04        043XX S WABASH AVE  820   
3   7903289    HT133522  2010-12-30      083XX S KINGSTON AVE  840   
4  10402076    HZ138551  2016-02-02           033XX W 66TH ST  820   

  PRIMARY_TYPE                    DESCRIPTION          LOCATION_DESCRIPTION  \
0        THEFT                  FROM BUILDING            SMALL RETAIL STORE   
1        THEFT                 $500 AND UNDER                         OTHER   
2        THEFT                 $500 AND UNDER  NURSING HOME/RETIREMENT HOME   
3        THEFT  FINANCIAL ID THEFT: OVER $300                     RESIDENCE   
4        THEFT                 $500 AND UNDER                         ALLEY   

   ARREST  DOMESTIC  ...  DISTR

Problem 1
Find the total number of crimes recorded in the CRIME table.

In [16]:
import sqlite3
import pandas as pd

# Connect to FinalDB.db
conn = sqlite3.connect("FinalDB.db")

# Query to count total crimes
query = "SELECT COUNT(*) AS total_crimes FROM CHICAGO_CRIME_DATA;"
result = pd.read_sql(query, conn)

# Display result
print(result)

# Close connection
conn.close()



   total_crimes
0           533


Problem 2
List community area names and numbers with per capita income less than 11000

In [17]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect("FinalDB.db")

# SQL Query to filter community areas
query = """
SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PER_CAPITA_INCOME 
FROM CENSUS_DATA 
WHERE PER_CAPITA_INCOME < 11000;
"""

# Run the query and load results into a DataFrame
result = pd.read_sql(query, conn)

# Display the result
print(result)

# Close the database connection
conn.close()


   COMMUNITY_AREA_NUMBER COMMUNITY_AREA_NAME  PER_CAPITA_INCOME
0                   26.0  West Garfield Park              10934
1                   30.0      South Lawndale              10402
2                   37.0         Fuller Park              10432
3                   54.0           Riverdale               8201


Problem 3
List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)


In [18]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect("FinalDB.db")

# SQL query to fetch case numbers for crimes involving minors
query = """
SELECT CASE_NUMBER 
FROM CHICAGO_CRIME_DATA 
WHERE LOWER(PRIMARY_TYPE) LIKE '%minor%' 
   OR LOWER(DESCRIPTION) LIKE '%minor%' 
   OR LOWER(PRIMARY_TYPE) LIKE '%juvenile%' 
   OR LOWER(DESCRIPTION) LIKE '%juvenile%';
"""

# Execute query and load results into a DataFrame
result = pd.read_sql(query, conn)

# Display results
print(result)

# Close the connection
conn.close()



  CASE_NUMBER
0    HL266884
1    HK238408


Problem 4
List all kidnapping crimes involving a child?


In [19]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect("FinalDB.db")

# SQL query to filter kidnapping cases involving a child
query = """
SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION 
FROM CHICAGO_CRIME_DATA 
WHERE LOWER(PRIMARY_TYPE) = 'kidnapping' 
AND LOWER(DESCRIPTION) LIKE '%child%';
"""

# Execute query and load results into a DataFrame
result = pd.read_sql(query, conn)

# Display results
print(result)

# Close connection
conn.close()


  CASE_NUMBER PRIMARY_TYPE               DESCRIPTION
0    HN144152   KIDNAPPING  CHILD ABDUCTION/STRANGER


Problem 5
List the kind of crimes that were recorded at schools. (No repetitions)

In [26]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect("FinalDB.db")

# SQL query to get distinct crime types at schools
query = """
SELECT DISTINCT PRIMARY_TYPE 
FROM CHICAGO_CRIME_DATA 
WHERE LOWER(LOCATION_DESCRIPTION) LIKE '%school%';
"""

# Execute query and load results into a DataFrame
result = pd.read_sql(query, conn)

# Display results
print(result)

# Close connection
conn.close()


             PRIMARY_TYPE
0                 BATTERY
1         CRIMINAL DAMAGE
2               NARCOTICS
3                 ASSAULT
4       CRIMINAL TRESPASS
5  PUBLIC PEACE VIOLATION


Problem 6
List the type of schools along with the average safety score for each type.¶

In [28]:
%%sql
PRAGMA table_info(CHICAGO_PUBLIC_SCHOOLS);


 * sqlite:///FinalDB.db
Done.


cid,name,type,notnull,dflt_value,pk
0,School_ID,INTEGER,0,None,0
1,NAME_OF_SCHOOL,TEXT,0,None,0
2,"Elementary, Middle, or High School",TEXT,0,None,0
3,Street_Address,TEXT,0,None,0
4,City,TEXT,0,None,0
5,State,TEXT,0,None,0
6,ZIP_Code,INTEGER,0,None,0
7,Phone_Number,TEXT,0,None,0
8,Link,TEXT,0,None,0
9,Network_Manager,TEXT,0,None,0


In [30]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect("FinalDB.db")

# SQL query to get the type of schools and their average safety score
query = """
SELECT NAME_OF_SCHOOL, AVG(SAFETY_SCORE) AS AVG_SAFETY_SCORE
FROM CHICAGO_PUBLIC_SCHOOLS
GROUP BY NAME_OF_SCHOOL;
"""

# Execute query and load results into a DataFrame
result = pd.read_sql(query, conn)

# Display results
print(result)

# Close connection
conn.close()


                                        NAME_OF_SCHOOL  AVG_SAFETY_SCORE
0                                 A.N. Pritzker School              59.0
1                    Abraham Lincoln Elementary School              99.0
2    Adam Clayton Powell Paideia Community Academy ...              54.0
3                  Adlai E Stevenson Elementary School              61.0
4                      Agustin Lara Elementary Academy              56.0
..                                                 ...               ...
561         Williams Preparatory Academy Middle School              49.0
562           Wilma Rudolph Elementary Learning Center               NaN
563                Wolfgang A Mozart Elementary School              41.0
564               Woodlawn Community Elementary School              70.0
565                 World Language Academy High School              51.0

[566 rows x 2 columns]


Problem 7
List 5 community areas with highest % of households below poverty line

In [31]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect("FinalDB.db")

# SQL query to get the top 5 community areas with highest % of households below the poverty line
query = """
SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY
FROM CENSUS_DATA
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
LIMIT 5;
"""

# Execute query and load results into a DataFrame
result = pd.read_sql(query, conn)

# Display results
print(result)

# Close connection
conn.close()


  COMMUNITY_AREA_NAME  PERCENT_HOUSEHOLDS_BELOW_POVERTY
0           Riverdale                              56.5
1         Fuller Park                              51.2
2           Englewood                              46.6
3      North Lawndale                              43.1
4  East Garfield Park                              42.4


Problem 8
Which community area is most crime prone? Display the coumminty area number only.


In [32]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect("FinalDB.db")

# SQL query to get the community area number with the most crimes
query = """
SELECT COMMUNITY_AREA_NUMBER
FROM CHICAGO_CRIME_DATA
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(*) DESC
LIMIT 1;
"""

# Execute query and load results into a DataFrame
result = pd.read_sql(query, conn)

# Display results
print(result)

# Close connection
conn.close()


   COMMUNITY_AREA_NUMBER
0                   25.0


Problem 9
Use a sub-query to find the name of the community area with highest hardship index

In [33]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect("FinalDB.db")

# SQL query with a sub-query to find the community area with the highest hardship index
query = """
SELECT COMMUNITY_AREA_NAME
FROM CENSUS_DATA
WHERE HARDSHIP_INDEX = (
    SELECT MAX(HARDSHIP_INDEX)
    FROM CENSUS_DATA
);
"""

# Execute query and load results into a DataFrame
result = pd.read_sql(query, conn)

# Display results
print(result)

# Close connection
conn.close()


  COMMUNITY_AREA_NAME
0           Riverdale


Problem 10
Use a sub-query to determine the Community Area Name with most number of crimes?

In [34]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect("FinalDB.db")

# SQL query with a sub-query to find the community area with the most crimes
query = """
SELECT COMMUNITY_AREA_NAME
FROM CENSUS_DATA
WHERE COMMUNITY_AREA_NUMBER = (
    SELECT COMMUNITY_AREA_NUMBER
    FROM CHICAGO_CRIME_DATA
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(*) DESC
    LIMIT 1
);
"""

# Execute query and load results into a DataFrame
result = pd.read_sql(query, conn)

# Display results
print(result)

# Close connection
conn.close()


  COMMUNITY_AREA_NAME
0              Austin
